# Formalization of the problem

## 1. Notations for reading time, free-reading time and total reading time
Let $R_F$ be the reading time for fiction books, $R_H$ the reading time for self-help books, $R$ the total reading time. Then the total reading time is the sum of the two:

\begin{equation*}
    R = R_F + R_H
\end{equation*}

For simplicity, we assume the expected demand fiction and self-help books is only modified by the free reading time (nothing else changes at the same time), namely $F_F$ and $F_H$. Hence we have

\begin{equation*}
    R = 30 * f(F_F, F_H) + 20 * g(F_F, F_H)
\end{equation*}

## 2. Optimizing the expected total reading time

We have a budget for free time of 120 minutes by customer, denoted $T$, that is to say $F_F + F_H \leq T$. After a little bit of EDA, it seems the more free-time, the more reading, so **we assume we use the whole budget**, ie $F_F + F_H = 1000000$, ie $F_H = T - F_F$, which reduces the dimensionality of the optimization problem, without ending up on a suboptimal equilibrium, as we'll confirm later on. Let **$p$ be the proportion of the budget** we allocate for free fiction book reading (then $F_F = p * T$ and $F_F = (1-p) * T$). We can simply formalize the first optimization objective as:

\begin{align*}
\max_{p} & E(f(p * T, (1-p) * T) + E(g(p * T, (1-p) * T))\\
\textrm{s.t.} & 0 \leq p \leq 1 \\
\end{align*} 

## 3. Optimizing the Sharpe ratio
To every coordinate $(F_F, F_H)$ in the space of possible free-reading time allocations, we can associate an expected total reading $E(R((F_F, F_H))$, that is the total reading time for a customer when giving him or her $(F_F, F_H)$ free reading time. But reading time at this point isn't deterministic. The exploratory data analysis suggests for each point in that space the **distribution of reading time is a gaussian**. Hence we can assume every point in that space has not only an expected reading time, but also a **standard deviation**, $std(R((F_F, F_H))$.

We might want to take into account that variability, maybe account for the **risk of the investment**, and instead of optimizing only the expected reading time, normalize it by its standard deviation (an indicator known as Sharpe ratio). The optimization problem for the second objective is:

\begin{align*}
\max_{p} & \frac{E(f(p * T, (1-p) * T) + E(g(p * T, (1-p) * T))}
{\sqrt{V(R(p)}}
\\
\textrm{s.t.} & 0 \leq p \leq 1 \\
\end{align*} 

In the modeling work, we use that:
\begin{equation*}
V(ax + bY) = a^2 V(X) + b^2 V(Y) + 2ab*\text{cov}(X, Y)
\end{equation*}
Though as we will see, it is not always easy to estimate $\text{cov}(R_F, R_H)$.

## 4. Using a surrogate model
The data we have give some strong indications to estimate the gaussian law for reading time over 12 point in the budget allocation space. However, we have no explicit formulation for that law elsewhere in the space. To solve the optimization problems, we must first modelize $R$ over the space and be able to estimate its expected value and standard deviation everywhere.

To do so, we use **surrogate models**. We used the following models:
- Random forests
- Support Vector Regressions
- Gaussian processes (technique known as kringing)

Because we only have data about 12 allocations, we sticked to **simple models** as much as possible when choosing the hyperparameters and did little tuning of hyperparameters, although this could be an area for further enhancements.

Once we have an estimator for the expected reading $\hat{E(R)}$ and standard deviation $\hat{sd(R)}$, we can solve the following optimization problems:
\begin{align*}
\max_{p} & \hat E(f(p * T, (1-p) * T) + \hat E(g(p * T, (1-p) * T))
\\
\textrm{s.t.} & 0 \leq p \leq 1 \\
\end{align*} 

and 

\begin{align*}
\max_{p} & \frac{\hat E(f(p * T, (1-p) * T) + \hat E(g(p * T, (1-p) * T))}
{\sqrt{\hat V(R(p)}}
\\
\textrm{s.t.} & 0 \leq p \leq 1 \\
\end{align*} 

## 5. Main difficulties and choices
### a. Modeling the reading times separately or together?
We want to optimize the expected total reading time, but there are two approaches for the modeling part:
1. **Model separately.** We first model the reading times for fiction and self-help, and then sum them to compute the associated total reading time.
2. **Model the total reading time.** We model the total reading time and use the estimator directly to maximize reading time.

Insights:
- When modeling the times separately, we might ignore some of the interactions we observed in the Exploratory Data Analysis.
- When modeling the total time directly, we get less business insights on why a certain allocation might be better than another.
- In practice, we implemented both method and found they have comparable estimations of total reading time.

We implemented both approaches and believe that:
- to understand how free reading time increases actual reading, the approaches that model reading times separately are more insightful.
- to predict the Sharpe ratio, the approaches that model total reading time directly are more reliable.

### c. Estimating the variance of the total reading time
#### Heteroscedasticity
The exploratory data analysis suggests the variance isn't constant accross the whole space but it depends on the free time allocation: it is **heteroscedastic**. That is something we need to bear in mind when estimating the variance with a gaussian process. We chose to use the implementation of the gaussian process estimator from the Python library smt because it accounts for heteroscedasticity and noise and exploits the repetition of observations in the 12 points we know.

It is worth noting heteroscedasticity is the whole reason the second optimization program is different from the first one.

#### Variance of the estimator vs variance of the variable
Most of the time, the methods implemented in the function we use for the modeling part estimate the variance of the estimator, not an estimation of the variance of the noise in any given point of the space.

In most notebooks, we used the variance of the variable. To do so, we did a simple linear interpolation with the point we knew of. Otherwise, when we used the variance of the estimator, it was so huge outside of the 12 points we know it completely changed the surrogate model and only a point we knew about could have been the optimum, which is not the point of this project.


Estimation of the expected total reading time with confidence bounds on the estimate:
![](https://github.com/alize-papp/SurrogateModelForOptimization/blob/main/images/reading_with_confidence_bounds.png)

We can see the confidence bounds are pretty wide, except on the three points we know (where the total free time is 120 minutes). For this reason, when generalizing on the whole space, the Sharpe ratio is pretty low (because the std is pretty high) except on the point we know.

With variance of the variable |  with variance of the estimator        
:-------------------------:|:-------------------------:
![](https://github.com/alize-papp/SurrogateModelForOptimization/blob/main/images/std-of-the-variable.png) | ![](https://github.com/alize-papp/SurrogateModelForOptimization/blob/main/images/std-of-the-estimator.png)  